# [Facebook Prophet](https://github.com/facebook/prophet) and [NeuralProphet](https://github.com/ourownstory/neural_prophet) Comparison
By: Rayhan Ozzy Ertarto

The goal of this notebook is to compare the *expected values* forecasted by these two models and compare them against the actuals in order to calculate the performance metrics and define which model performs better using this time series dataset (Rainfall in Kemayoran BMKG Station, Central Jakarta)

Importing basic libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
np.random.seed(1234)

In [3]:
plt.style.use('ggplot')

Reading the time series

In [4]:
gsheetkey = "1QUan4wVK8yNaIDvLvQEXocTWch8xkjy8kvS53PAKt_U"

url=f'https://docs.google.com/spreadsheet/ccc?key={gsheetkey}&output=csv'
df_km = pd.read_csv(url)
df_km.head(10)

In [5]:
df_km.info()

In [6]:
df_km['RR'] = df_km['RR'].replace([8888.0],['NaN'])

In [7]:
df_km['RR'] = df_km['RR'].replace([9999.0],['NaN'])

In [8]:
df_km.head(10)

In [9]:
df_km.info()

In [10]:
df_km['RR'] = df_km['RR'].astype(float)
df_km.info()

In [11]:
df_km.head()

Check for Missing Values

In [12]:
df_km.isna().sum()

Fill Missing Values by Interpolation

In [13]:
df_km = df_km.interpolate()

In [14]:
df_km.isna().sum()

In [15]:
df_km.head(10)

In [16]:
df_km['Tanggal'] = pd.to_datetime(df_km['Tanggal'])
df_km.info()

In [17]:
# Renaming columns
df_km.rename(columns = {'Tanggal':'ds', 'RR':'y'}, inplace = True)
df_km.head()

In [18]:
df_km.tail()

In [19]:
df_km.set_index('ds').plot(figsize=(12,6))
plt.title('Time Series Plot')

## Prophet Model

In [20]:
!pip install prophet -q

In [21]:
from prophet import Prophet

In [22]:
m = Prophet(seasonality_mode='additive')

Using default settings, only the seasonality mode is set to *Additive*



In [23]:
m.fit(df_km)

In [24]:
future = m.make_future_dataframe(periods=60, freq='D')

In [25]:
future.tail(5)

In [26]:
forecast = m.predict(future)

In [27]:
forecast.tail()

In [28]:
m.plot(forecast);
plt.title("Forecast of the Time Series in the next 60 days")

In [29]:
m.plot_components(forecast);
print("Components of the time series:")

In [30]:
#p_forecast = forecast[forecast['ds']>'2022-02-18'][['ds','yhat_lower','yhat','yhat_upper']]
p_forecast = forecast[['ds','yhat_lower','yhat','yhat_upper']]
p_forecast.info()

In [31]:
plt.figure(figsize=(12,6))
plt.xticks(rotation=45)
plt.title("Detail of Forecast using Prophet")
plt.plot(p_forecast['ds'], p_forecast['yhat'], marker='.', c='navy')
plt.fill_between(p_forecast['ds'],p_forecast['yhat_lower'], p_forecast['yhat_upper'], alpha=0.1, color='cyan')

### Performance Metrics

In [32]:
from sklearn.metrics import mean_squared_error

In [33]:
df_km.info()

In [34]:
df_km_merge = pd.merge(df_km, forecast[['ds','yhat_lower','yhat_upper','yhat']],on='ds')
df_km_merge = df_km_merge[['ds','yhat_lower','yhat_upper','yhat','y']]
df_km_merge.head()

In [35]:
df_km_merge.tail()

In [36]:
prophet_mse = mean_squared_error(df_km_merge['y'], df_km_merge['yhat'])
prophet_rmse = np.sqrt(mean_squared_error(df_km_merge['y'], df_km_merge['yhat']))

In [37]:
print("Prophet MSE: {:.4f}".format(prophet_mse))
print("Prophet RMSE: {:.4f}".format(prophet_rmse))

## NeuralProphet

In [38]:
!pip install neuralprophet -q

In [39]:
from neuralprophet import NeuralProphet, set_random_seed

In [40]:
set_random_seed(42)

In [41]:
nm = NeuralProphet(seasonality_mode='additive')

In [42]:
nm.fit(df_km, freq='D')

In [43]:
n_future = nm.make_future_dataframe(df_km, periods=60, n_historic_predictions=len(df_km))
n_future

In [44]:
n_future.tail()

In [45]:
n_forecast = nm.predict(n_future)

In [46]:
n_forecast.info()

In [47]:
n_forecast.tail()

In [48]:
plt.figure(figsize=(12,6))
plt.xticks(rotation=45)
plt.title("Detail of Forecast using NeuralProphet")
plt.plot(n_forecast['ds'], n_forecast['yhat1'], marker='.', c='red')
plt.legend()

In [49]:
nm.plot(pd.concat([df_km, n_forecast], ignore_index=True));
plt.title("Forecast of the Time Series in the next 60 days")

In [50]:
nm.plot_components(pd.concat([df_km, n_forecast], ignore_index=True));

### Performance Metrics

In [51]:
n_forecast

In [52]:
n_forecast_merge = pd.merge(df_km, n_forecast[['ds','yhat1','residual1']],on='ds')
n_forecast_merge = n_forecast_merge[['ds','yhat1','residual1','y']]
n_forecast_merge.head()

In [53]:
n_prophet_mse = mean_squared_error(n_forecast_merge['y'], n_forecast_merge['yhat1'])
n_prophet_rmse = np.sqrt(mean_squared_error(n_forecast_merge['y'], n_forecast_merge['yhat1']))

In [54]:
print("Neural Prophet MSE: {:.4f}".format(n_prophet_mse))
print("Neural Prophet RMSE: {:.4f}".format(n_prophet_rmse))

In [55]:
print("Prophet MSE: {:.4f}".format(prophet_mse))
print("Prophet RMSE: {:.4f}".format(prophet_rmse))

In [56]:
n_prophet_mse - prophet_mse

In [57]:
n_prophet_rmse - prophet_rmse

In [58]:
plt.figure(figsize=(12,6))
plt.xticks(rotation=45)
plt.title("Models Comparison")
plt.plot(p_forecast['ds'], p_forecast['yhat'], marker='.', c='navy', label='Prophet')
plt.plot(n_forecast['ds'], n_forecast['yhat1'], marker='.', c='red', label='NeuralProphet')
plt.legend()

In [59]:
pd.DataFrame({'metrics':['MSE','RMSE'],
              'Prophet ':[prophet_mse, prophet_rmse],
              'Neural Prophet':[n_prophet_mse, n_prophet_rmse]
             })

## Final Comments

*   At least for this particular dataset and using the default arguments,  the **NeuralProphet** model scored a **MSE** of **326.223127** and **RMSE** of **18.061648** whereas the **Prophet** model scored a **MSE** of **332.112837** and **RMSE** of **18.223963**, a **5.889710168371835 and 0.16231530824208207 difference of MSE and RMSE respectively** compared against the first model.